# Advanced Lane Finding - Project 4 of Udacity's Self-Driving Car Nanodegree

1. Camera Calibration
2. Distortion correction
3. Image inspection
4. Perspective correction

## 1. Camera Calibration

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import ntpath

%matplotlib inline

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

col_count = 3
row_count = 7
fig = plt.figure(figsize=(16,32))

index = 0

# Step throuagh the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        sp = fig.add_subplot(row_count, col_count, index+1)
        plt.imshow(img)
        index += 1

plt.show()

In [ ]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

## 2. Distortion Correction

In [ ]:
images = glob.glob('camera_cal/calibration*.jpg')

fig = plt.figure(figsize=(16,32))
index = 0

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    sp = fig.add_subplot(row_count, col_count, index+1)
    plt.imshow(img)
    index += 1

plt.show()

## 3. Example image inspection

In [ ]:
images = ['test_images/test1.jpg', 'test_images/test2.jpg', 'test_images/test3.jpg',
          'test_images/test4.jpg', 'test_images/test5.jpg', 'test_images/test6.jpg',
          'test_images/straight_lines1.jpg', 'test_images/straight_lines2.jpg']

example_images = images

col_count = 2
row_count = 8
fig = plt.figure(figsize=(20,60))

index = 0

example_image = None

# Step through the list and search for chessboard corners
for fname in example_images:
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    sp = fig.add_subplot(row_count, col_count, index*2+1)
    plt.imshow(img)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    sp = fig.add_subplot(row_count, col_count, index*2+2)
    plt.imshow(img)
    index += 1

plt.show()

# 4. Perspective correction

In [ ]:
example_image = cv2.imread(example_images[6])
example_image = cv2.cvtColor(example_image, cv2.COLOR_BGR2RGB)
example_image = cv2.undistort(example_image, mtx, dist, None, mtx)

img_size = (example_image.shape[1],example_image.shape[0])

# defines the perspective of the camera image by defining points near the front of the car and close the the
# center of the image
relation_factor = 13.5/1.92
front_perspective_div = 3.5
back_perspective_div = front_perspective_div*relation_factor
front_perspective_y_perc = 0.92
back_perspective_y_perc = 0.63

# trapez points in order bottom left, top left, top right, bottom right (front, back, back, front)
src = [(int(img_size[0]/2-img_size[0]/front_perspective_div), int(img_size[1]*front_perspective_y_perc)), 
       (int(img_size[0]/2-img_size[0]/back_perspective_div), int(img_size[1]*back_perspective_y_perc)), 
       (int(img_size[0]/2+img_size[0]/back_perspective_div), int(img_size[1]*back_perspective_y_perc)),
       (int(img_size[0]/2+img_size[0]/front_perspective_div), int(img_size[1]*front_perspective_y_perc))]

# paint trapez into the image
image_copy = np.copy(example_image)

for index in range(4):
    pa = src[index]
    pb = src[(index+1)%4]
    cv2.line(image_copy, pa, pb, (255,0,0), 4)

fig = plt.figure(figsize=(20,20))
plt.imshow(image_copy)

margin_factor = 6

dst = [(img_size[0]//margin_factor, img_size[1]), 
       (img_size[0]//margin_factor, 0), 
       (img_size[0]-img_size[0]//margin_factor, 0), 
       (img_size[0]-img_size[0]//margin_factor, img_size[1]), 
       ]

src = np.array(src, dtype=np.float32)
dst = np.array(dst, dtype=np.float32)

tmx = cv2.getPerspectiveTransform(np.array(src), np.array(dst))

plt.show()

width = image_copy.shape[1]
height = image_copy.shape[0]

warped = cv2.warpPerspective(example_image, tmx, (width, height))

fig = plt.figure(figsize=(20,20))
plt.imshow(warped)

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

project_video = "project_video.mp4"

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(project_video))

### Video of the project video after distortion and perspective correction

In [ ]:
def process_image(image):
    undistorted = cv2.undistort(image, mtx, dist, None, mtx)
    warped = cv2.warpPerspective(undistorted, tmx, (width, height))
    return warped
    
from_above_video = 'test_videos_output/from_above.mp4'

white_output = from_above_video
clip1 = VideoFileClip(project_video)
white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(from_above_video))

## Highlighting lanes

In [ ]:
col_count = 2
row_count = 8
fig = plt.figure(figsize=(20,60))
index = 0

# Step through the list and search for chessboard corners
for fname in example_images:
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    sp = fig.add_subplot(row_count, col_count, index+1)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    plt.title(ntpath.basename(fname))
    warped = cv2.warpPerspective(img, tmx, (width, height))
    plt.imshow(warped, 'gray')
    index += 1

plt.show()

In [ ]:
def hls_threshold_mask(image, thresh=(95,255)):
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    
    s = hls[:,:,2]
    
    binary_output = np.zeros_like(s)
    binary_output[(s>=thresh[0]) & (s<=thresh[1])] = 1

    return binary_output

def sobel_mag_mask(image, thresh=(10,255), kernel_size=3):
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=kernel_size)
    sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=kernel_size)
    abs_sobelxy = np.sqrt(sobel_x**2 + sobel_y**2)
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    eight_bit = np.uint8(255*abs_sobelxy/np.max(abs_sobelxy))
    # 5) Create a binary mask where mag thresholds are met
    # 6) Return this mask as your binary_output image
    binary_output = np.zeros_like(abs_sobelxy)
    binary_output[(eight_bit >= thresh[0]) & (eight_bit <= thresh[1])] = 1    

    return binary_output

def sobel_dir_mask(image, thresh=(0.7, 1.3), kernel_size=13):
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=kernel_size)
    sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=kernel_size)
    abs_x = np.abs(sobel_x)
    abs_y = np.abs(sobel_y)
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    sob_dir = np.arctan2(abs_y, abs_x)
    # 5) Create a binary mask where direction thresholds are met    
    # 6) Return this mask as your binary_output image
    binary_output = np.zeros_like(sob_dir)
    binary_output[(sob_dir>=thresh[0]) & (sob_dir<=thresh[1])] = 1
    
    return binary_output

def create_binary_mask(image):
    hls_thresh = hls_threshold_mask(image)
    sobel_mag = sobel_mag_mask(image)
    sobel_dir = sobel_dir_mask(image)
    
    binary_output = np.zeros_like(sobel_dir)
    binary_output[(hls_thresh==1)] = 1.0
#    binary_output[(hls_thresh==1) | ((sobel_mag==1) & (sobel_dir==1))] = 1
    ## binary_output[(hls_thresh==1)] = 1
    
    return binary_output

col_count = 2
row_count = 8
fig = plt.figure(figsize=(20,60))
index = 0

# Step through the list and search for chessboard corners
for fname in example_images:
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    sp = fig.add_subplot(row_count, col_count, index+1)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    # img = sobel_mag_mask(img)
    img = create_binary_mask(img)
    plt.title(ntpath.basename(fname))
    warped = cv2.warpPerspective(img, tmx, (width, height))
    plt.imshow(warped, 'gray')
    index += 1

plt.show()

In [ ]:
col_count = 2
row_count = 8
fig = plt.figure(figsize=(20,60))
index = 0

# Step through the list and search for chessboard corners
for fname in example_images:
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    sp = fig.add_subplot(row_count, col_count, index+1)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    # img = sobel_mag_mask(img)
    img = create_binary_mask(img)
    plt.title(ntpath.basename(fname))
    warped = cv2.warpPerspective(img, tmx, (width, height))
    histogram = np.sum(warped[img.shape[0]//2:,:], axis=0)
    plt.plot(histogram)
    index += 1

plt.show()

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

class lane_finder:
    def __init__(self):
        self.last_base_left = None
        self.last_base_right = None
        self.hist_minimum_thresh = 30
        self.hist_perf_thresh = 60
        self.lane_size = None
        self.perfect_pix_win = 140
    
    def find_lanes_using_window(self, img):
        sp = fig.add_subplot(row_count, col_count, index+1)
        img = cv2.undistort(img, mtx, dist, None, mtx)
        binary_warped = create_binary_mask(img)
        binary_warped = cv2.warpPerspective(binary_warped, tmx, (width, height))    

        histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255

        # Find the peak of the left and right halves of the histogram
        # These will be the starting point for the left and right lines
        midpoint = np.int(histogram.shape[0]//2)
                
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint
        
        if histogram[leftx_base]<=self.hist_minimum_thresh and self.last_base_left is not None:
            leftx_base = self.last_base_left
        else:
            self.last_base_left = leftx_base
        
        if histogram[rightx_base]<=self.hist_minimum_thresh and self.last_base_right is not None:
            rightx_base = self.last_base_right
        else:
            self.last_base_right = rightx_base

        # if we can right now perfectly detect both lanes recalibrate the lane size
        if histogram[leftx_base]>=self.hist_perf_thresh and histogram[rightx_base]>=self.hist_perf_thresh:
            self.lane_size = rightx_base-leftx_base

        # Choose the number of sliding windows
        nwindows = 9
        # Set height of windows
        window_height = np.int(binary_warped.shape[0]//nwindows)
        # Identify the x and y positions of all nonzero pixels in the image
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Current positions to be updated for each window
        leftx_current = leftx_base
        rightx_current = rightx_base
        # Set the width of the windows +/- margin
        margin = 100
        # Set minimum number of pixels found to recenter window
        minpix = 50
        # Create empty lists to receive left and right lane pixel indices
        left_lane_inds = []
        right_lane_inds = []
        
        dominant_l = False
        dominant_r = False

        # Step through the windows one by one
        for window in range(nwindows):
            # Identify window boundaries in x and y (and right and left)
            win_y_low = binary_warped.shape[0] - (window+1)*window_height
            win_y_high = binary_warped.shape[0] - window*window_height
            win_xleft_low = leftx_current - margin
            win_xleft_high = leftx_current + margin
            win_xright_low = rightx_current - margin
            win_xright_high = rightx_current + margin
            
            color_l = (0,255,0)
            color_r = (0,255,0)
            
            # Identify the nonzero pixels in x and y within the window
            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
            (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
            (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
            # Append these indices to the lists
            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)
            # If you found > minpix pixels, recenter next window on their mean position
            if len(good_left_inds) > minpix:
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            if len(good_right_inds) > minpix:        
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
                
            if len(good_left_inds)>self.perfect_pix_win:
                dominant_l = True
            else:
                dominant_l = False
                
            if len(good_right_inds)>self.perfect_pix_win:
                dominant_r = True
            else:
                dominant_r = False

            if dominant_l:
                color_l = (255,0,0)                
            if dominant_r:
                color_r = (255,0,0)                            

            # Draw the windows on the visualization image
            cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),
            color_l, 2) 
            cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),
            color_r, 2) 
                

        # Concatenate the arrays of indices
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)

        # Extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds] 

        # Fit a second order polynomial to each
        left_fit = None
        right_fit = None

        # Generate x and y values for plotting
        ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )

        if leftx.shape[0]!=0 and lefty.shape!=0:
            left_fit = np.polyfit(lefty, leftx, 2)
            left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        if rightx.shape[0]!=0 and righty.shape!=0:
            right_fit = np.polyfit(righty, rightx, 2)
            right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

        out_img = out_img.astype(np.ubyte)

        return out_img

for cur_fn in example_images:
    img = cv2.imread(cur_fn)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    lf = lane_finder()
    out_img = lf.find_lanes_using_window(img)
    
    fig = plt.figure(figsize=(12,8))
    plt.imshow(out_img)
#    plt.plot(left_fitx, ploty, color='yellow')
#    plt.plot(right_fitx, ploty, color='yellow')
    plt.xlim(0, 1280)
    plt.ylim(720, 0)
    plt.show()    


In [ ]:
lf = lane_finder()

def process_image(image):
    warped = lf.find_lanes_using_window(image)
    return warped
    
find_lanes_raw = 'test_videos_output/find_lanes_raw.mp4'

white_output = find_lanes_raw
clip1 = VideoFileClip(project_video)
white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(find_lanes_raw))